In [2]:
from corenlpsf import StanfordNLP
from sklearn.externals import joblib
from collections import defaultdict, Counter
sNLP = StanfordNLP()
def countpos(filenames):
    pos_count = Counter()
    pos_dict = defaultdict(list)
    for fname in filenames:
        with open(fname, "r") as f:
            for line in f:
                wds, pos = zip(*sNLP.pos(line.strip()))
                pos_count[str(pos)] += 1
                pos_dict[str(pos)].extend([" ".join(wds)])
    return pos_dict, pos_count
fname = "/media/data/hotel/kdd_merge_posneg.txt"
pos_dict, pos_count = countpos([fname])
joblib.dump([pos_count,pos_dict], "/media/data/hotel/pos_count_dict.pkl")
# filenames = ['/media/data/booking.com/booking.negative.sorted.tkn.txt', '/media/data/booking.com/booking.positive.sorted.tkn.txt']
# pos_dict, pos_count = countpos(filenames)
# joblib.dump([pos_count,pos_dict], "/media/data/booking.com/pos_count_dict.pkl")
# pos_count,pos_dict = joblib.load("/media/data/booking.com/pos_count_dict.pkl")

['/media/data/hotel/pos_count_dict.pkl']

In [4]:
pos_count.most_common(10)

[("('JJ', 'NN', '.')", 1637),
 ("('DT', 'NN', 'VBD', 'JJ', '.')", 579),
 ("('PRP', 'MD', 'RB', 'VB', 'RB', 'RB', '.')", 566),
 ("('RB', 'JJ', '.')", 498),
 ("('RB', 'VBN', '.')", 459),
 ("('RB', 'JJ', 'NN', '.')", 441),
 ("('NNP', 'NN', '.')", 428),
 ("('MD', 'RB', 'VB', 'RB', 'RB', '.')", 364),
 ("('DT', 'NN', 'VBZ', 'JJ', '.')", 362),
 ("('NN', 'VBD', 'JJ', '.')", 310)]

In [6]:
pos_dict["('PRP', 'MD', 'RB', 'VB', 'RB', 'RB', '.')"][:10]

['I will never stay here again .',
 'I will never stay there again .',
 'I will never stay here again .',
 'I will never stay here again .',
 'I would never stay here again !',
 'I would not stay there again .',
 'I would never stay there again .',
 'I would not stay there again .',
 'I would never stay there again .',
 'I will not stay there again .']

In [15]:
from gensim import corpora
from six import iteritems

    
class MyData(object):
    def __init__(self, filenames):
        self.filenames = filenames
    
    def __iter__(self):
        for fname in self.filenames:
            with open(fname, "r") as f:
                for line in f:
                    yield line.strip().lower().split()
       
    
filenames = ['/media/data/booking.com/booking.negative.sorted.tkn.txt', 
             '/media/data/booking.com/booking.positive.sorted.tkn.txt']
stoplist = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", 
            "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", 
            "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
            "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how",
            "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
            "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", 
            "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
            "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", 
            "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through",
            "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", 
            "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", 
            "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]


corpus = MyData(filenames)   
dictionary = corpora.Dictionary(sent for sent in corpus)
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()
dictionary.save('/tmp/booking.dictionary.dict')
print(dictionary)

In [20]:
class MyCorpus(object):
    def __init__(self, dictionary, corpus):
        self.dictionary = dictionary
        self.corpus = corpus
        
    def __iter__(self):
        for sent in self.corpus:
            # assume there's one document per line, tokens separated by whitespace
            yield self.dictionary.doc2bow(sent)
            
bowvecs = []
vecgen = MyCorpus(dictionary, corpus)
for vec in vecgen:
    bowvecs.append(vec)
corpora.MmCorpus.serialize('/tmp/booking.corpus.mm', bowvecs)

In [29]:
from gensim import models, similarities
tfidf = models.TfidfModel(bowvecs) # step 1 -- initialize a model
corpus_tfidf = tfidf[bowvecs]

In [50]:
bowvecs[109090]

[(733, 1), (1998, 1)]

In [39]:
dictionary.get(20)

'spa'

In [40]:
corpus_tfidf[0]

[(0, 0.02218453380358644), (1, 0.8372404062115201), (2, 0.54638479908091)]

In [43]:
tfidf[[(0, 1), (1, 1), (2, 1)]]

[(0, 0.02218453380358644), (1, 0.8372404062115201), (2, 0.54638479908091)]

In [75]:
tfidf[dictionary.doc2bow("awesome room food awesome awesome awesome".split())]

[(53, 0.09042548516812363),
 (69, 0.1971464786545802),
 (1234, 0.9761949075805552)]

In [72]:
dictionary[100]

'nice'